## EE 502 P: Analytical Methods for Electrical Engineering
    
# Homework 10: Review
## Due 15 December, 2019 at 11:59 PM

## Name: Kyle Hadley

Copyright &copy; 2019, University of Washington

<hr>

**Instructions**: Choose **<u>one</u>** of the following problems. Solve the problem and then write up your solution in a stand alone Jupyter Notebook. Your notebook should have the following elements:

- Problem statement
- Mathematical description of the solution
- Executable code, commented, clear code

You will be graded on how well your notebook reads like a nicely formated, well written report. You must:

- Write mathematical descriptions using complete sentences, paragraphs, and LaTeX formulas. 
- Comment your code as necessary, with a description of what each function does and all major steps.
- Label plots axes, use legends, and use plot titles. 

# Problem Statement: Hallucinating the Constitution

Consider the constitution of the United States:

> https://www.usconstitution.net/const.txt .

This document contains upper- and lower-case letters, numbers, and basic punctuation. 

**One letter prediction:**

1. Find the set of all characters used in the document. Call the number of characters $n$. 
2. Create an $n \times n$ matrix whose $i,j$ entry is the probability that the next character is $j$ given that the current character is $i$. Estimate this probability by looking at all occurrences of character $i$ in the document and the number of times character $j$ immediately follows it. 
3. Simulate this system as a Markov chain that starts with an arbitrary capital letter and continues until it gets to a space. Produce $100$ random "words" this way. How many of them are actual works (use a [Scrabble dictionary](https://scrabble.hasbro.com/en-us/tools#dictionary) if you are not certain whether a given sequence is a word). 

**Two letter prediction:**

1. Create an $n \times n \times n$ tensor whose $i,j,k$ entry is the probability that the next character is $k$ given that the current character is $j$ and the previous character is $i$. Use the document to empirically find these probabilities. 
2. Use this model to construct random words. 

**Sentence prediction:**

Do a one word prediction, but use all the unique *words* in the document. Hallucinate sentences. Consider a punctuation mark as a word.

**Notes:** Use `open` and `file.read` to read in the file as a string. For the sentence. Use `replace` to add space before punctuation and then `split()` to turn the string into a list. Use a `DiGraph` from the `networkx` library to store the data. Note that you can make weighted edges by adding data to the edges, as in [this document](https://networkx.github.io/documentation/stable/auto_examples/drawing/plot_weighted_graph.html).

---

# Data Input and Structure

The first step is to import the necessary libraries that will be used for the problem statement. Then seed the random function with a value of $1$, such that our solutions can be repeated.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sys

random.seed(1)
np.random.seed(1)

# np.set_printoptions(threshold=sys.maxsize)

The text version of the US Constituion is saved as a local file called `const.txt`. This file is read-in and saved to a string (`text_file`) containing the entire text of the US Constitution.

A series of `replace` calls are made to add spaces around non-alphanumeric characters that appear within the Constitution. As an example, every comma has a space added before it. By adding these additional spaces, the `split` function can be called to generate an array, $W$, of the words in the text file. This array will be used in the 3rd part of the solution.

From $W$, the set of all unique words that appear in the US Constituion $X$ is defined as

$$ X = \{word : word \in \text{US Consitution}\}.$$

The set $X$ is calculating using numpy's `unique(array)` function which returns the unique values for the provided numpy array.

From the `text_file` string, an array containing all characters in the text file, $A$, is generated using the function `list(string)`. This function transforms a string into a list of characters (which are then converted into a numpy array).

From $A$, the set of all unique characters that appear in the US Constituion $C$ is defined as

$$ C = \{character : character \in \text{US Consitution}\}.$$

The `unique` function is used again to create $C$.

With this operation, we find that are $69$ elements within the set $C$ - this value is saved as $n$. The elements and the count (with truncation) of the sets are printed to console below.

In [2]:
# Read in the US Consitution from the appropriate text file and save the
# contents of the file as a single string.

filename = "data\const.txt"

f = open(filename, "r")
if f.mode == 'r':
    file_text = f.read()

f.close()

# For punctuation, spaces either before or after (depending on the character)
# such that the split function will be able to capture punctuation as
# separate "words".

file_text = file_text.replace(".", " .")
file_text = file_text.replace(",", " ,")
file_text = file_text.replace("(", "( ")
file_text = file_text.replace(")", " )")
file_text = file_text.replace("\n", " ")
file_text = file_text.replace(":", " :")
file_text = file_text.replace(";", " ;")
file_text = file_text.replace("\"", " \" ")

# Convert the 'file_text' string into a series of numpy arrays:
# W - Ordered array of words as they appear in the text file.
# X - Array of unique words that appear in the text file.
# A - Ordered array of characters as they appear in the text file.
# C - Array of unique characters that appear in the text file.

W = np.array(file_text.split())
X = np.unique(W)

A = np.array(list(file_text))
C = np.unique(A)

n = len(C)

# Print the contents and size of X and C. Print the size of array A.

print('X =', X)
print('|X| (unique words in text) = ', len(X), "\n")

print('C =', C)
print('|C| (unique characters in text) = n =', len(C), "\n")

print('A (total # of characters in text) =', len(A))

X = ['"' '(' ')' ... 'written' 'year' 'years']
|X| (unique words in text) =  1351 

C = [' ' '"' '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':'
 ';' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q'
 'R' 'S' 'T' 'U' 'V' 'W' 'Y' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k'
 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
|C| (unique characters in text) = n = 69 

A (total # of characters in text) = 45737


In order to verify if a word is valid later on, an English dictionary is also read in as saved as `en_dict`. This dictionary includes over 479k English words, as found at the following link: https://github.com/dwyl/english-words.

In [3]:
# Read in the contents of the words alpha file - which will act
# as our english dictionary for determining whether a word is valid
# later on.

filename = "data\words_alpha.txt"

f = open(filename, "r")
if f.mode == 'r':
    file_text = f.read()

f.close()

en_dict = np.unique(np.array(file_text.split()))
print(en_dict)

['a' 'aa' 'aaa' ... 'zyzzogeton' 'zyzzyva' 'zyzzyvas']


---

# One Letter Prediction

For **one-letter prediction**, the first step is to generate a Stochastic Matrix $Q$ such that we can perform single letter predictions to generate random "words".

$Q$ is defined such that $Q_{i,j}$ defines the probability of the next character being $j$ given the current character is $i$. As $Q$ is a Stochastic Matrix, it is given that

- $Q_{i,j} \in [0,1]$.
- The sum of each row is one.

Using the definition of the Conditional Probability, the probability of the pairing of $(i, j)$ - character $i$ following by character $j$ - is defined as

$$Q_{i,j} = P[J\;|\;I] = \frac{P[I\cap J]}{P[I]}$$

where $I$ is the event of character $i$ appearing as the 1st character and $J$ is the event of character $j$ appearing as the 2nd character.

$P[I\cap J]$ and $P[I]$ are defined as

$$P[I\cap J] = \frac{f_{(i,j)}}{|A|}$$
$$P[I] = \frac{f_{i}}{|A|}$$

where $|A|$ is the total number of characters is the US Constitution and $f_x$ is the frequency of occurence of character or sequence $x$.

Simplifying for $P[J\;|\;I]$,

$$P[J\;|\;I] = \frac{f_{(i,j)}}{f_{i}}.$$

Thus, we see that

$$Q_{i,j} = \frac{f_{(i,j)}}{f_{i}}.$$

In order to associate each character with the indices in $Q$ (or other matrices), a dictionary $D$ can be created that associates each unique character $x$ with an index $i_x$. This dictionary is defined as

$$ D\,[x] = i_x.$$

For example,

$$ D\,[\,\text{'A'}\,] = 19$$
$$ D\,[\,\text{'0'}\,] = 7$$
$$ D\,[\,\text{'z'}\,] = 68.$$

A numpy array (i.e. matrix) $f$ is initialized with a size equal to $n \times n$, with each element $f_{(i,j)} = 0$. As the character array $A$ is iterated over, each iterated element $A_x$ and succeeding element $A_{x+1}$, $1$ is added to $f_{(i, j)}$ in which $i$ and $j$ corresponds to the indices of $A_x$ and $A_{x+1}$, respectively.

$f_i$ is equivalent to the sum of the $ith$ row of $f$, and thus $Q$ can be calculating using numpy's `divide` function to divie each element of $f$ by summation of its corresponding row.

The contents of $Q$ (with truncation) are printed to console.

In [4]:
# Create a dictionary / hash of the set C - this dictionary D can
# provide the index within our various matrices (Q, f, etc.).
# We can do this by calling D[a], in which 'a' is our character of interest.

D = {k: v for v, k in enumerate(C)}

# Calculate the count of character j following i within array / matrix A.
# For the last character x+1 = len(A), we will assume that it is followed
# by a space.

f = np.zeros((n, n))

for x in range(len(A)):
    i = D[A[x]]
    if (x + 1) >= len(A):
        j = D[" "]
    else:
        j = D[A[x+1]]

    f[i, j] = f[i, j] + 1

# Turn each value into a probability rather than a count. Numpy's divide
# function does a true divison of the inputs elementwise, excluding instances
# in which the sum of a row in f is equal to 0.
Q = np.divide(f, f.sum(axis=1)[:, None],
              out=np.zeros_like(f), where=f.sum(axis=1)[:, None] != 0)

print('Q =', Q)

Q = [[1.94633191e-02 2.30335138e-04 5.75837844e-04 ... 0.00000000e+00
  1.38201083e-03 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.35416667e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.24528302e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


From here, a while loop is created to simulate the system as a Markov chain starting with an arbitrary capital letter and continues until it gets a space or any other non-alpha character (i.e. punctuations and numbers).

Array `c_letters` which contains all of the capital letters that appear within the US Consitution - which, by observation, includes all letters except "X" and "Z".

The resulting words are saved to an array `words`.

The first step to creating a 'word' is to determine a random capital letter, using `random.choice`, and save to the variable `current_letter`. Next, the while loop is entered and iterates as long as `current_letter` is an alpha character.

Inside the while loop, the loop first appends the `current_letter` to the string `word` string, which will be the output of the while loop. A new letter is determined using `np.random.choice`; `np.random.choice` is capable of choosing a random character given a probability of occurence. In this case, the probability of occurence is defined by the row $Q_{i}$ where $i$ corresponds to the index of `current_letter`.

Once a `current_letter` is found that is not an alpha character, the while loop breaks and saves `word` to the array `words`.

This process is wrapped by a for loop to produce $100$ random 'words'. These words are outputted to display.

In [5]:
# Array of all the capital letters that appear within the US Constitution
c_letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J",
             "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T",
             "U", "V", "W", "Y"]

words = []

# Enter a for 100-iteration loop to generate 100 random 'words'.

for i in range(100):

    # Select a random letter from the array c_letters.
    # Iterate over the while loop until a non-alpha character is reached.
    current_letter = random.choice(c_letters)
    word = ""

    while current_letter.isalpha():
        # Add our current letter / character to our word. Select a new random letter using
        # our probability matrix Q.
        word = word + current_letter

        current_letter = C[np.random.choice(
            len(D), 1, p=Q[D[current_letter], :])][0]

    words.append(word)

In [6]:
# Keep a count of all valid and invalid words when checking
# against the english dictionary.
valid_count = 0
invalid_count = 0

# Iterate over all the generated words and check if they exist within
# our english dictionary array. If they exist, then output as valid
# and add to valid counter. If not, then output as invalid and add
# to invalid counter. Print results of counters to console.
for word in words:
    if word.upper() in [a.upper() for a in en_dict]:
        print(word, '- Valid')
        valid_count = valid_count + 1
    else:
        print(word, '- Invalid')
        invalid_count = invalid_count + 1

print('\nNumber of valid words =', valid_count)
print('Number of invalid words =', invalid_count)

Ell - Valid
Se - Valid
Calangen - Invalid
In - Valid
Ding - Valid
Pllumest - Invalid
Oall - Invalid
Prurilll - Invalid
Uny - Invalid
Mat - Valid
Gote - Valid
Da - Valid
Pr - Valid
Ale - Valid
Me - Valid
Nor - Valid
Thecanleshm - Invalid
Adin - Valid
Wrernthe - Invalid
Ofonven - Invalid
Indecesh - Invalid
Ye - Valid
Hotesh - Invalid
Sthtaith - Invalid
Distica - Invalid
Kieshes - Invalid
Alive - Valid
Ar - Valid
Aresiof - Invalid
Un - Valid
Re - Valid
Appacur - Invalid
Matarviedes - Invalid
Vachemby - Invalid
Gonshache - Invalid
No - Valid
Yer - Valid
Angr - Invalid
Quslll - Invalid
Honuthoponintretag - Invalid
Of - Valid
Prs - Valid
Re - Valid
Hereshitongrang - Invalid
Lanteshath - Invalid
Hort - Valid
Vomprshe - Invalid
Horsirs - Invalid
Of - Valid
Jond - Invalid
Andur - Invalid
Noitothallererered - Invalid
Reacudithe - Invalid
Und - Invalid
Dilll - Invalid
Faler - Invalid
Unthe - Invalid
Yege - Invalid
Juricr - Invalid
Der - Valid
Yesshivit - Invalid
Kishe - Invalid
Yer - Valid
Wendge

In a sample test run, below is the resulting output of the one-letter word prediction. The one-letter word prediction was able to generate $38$ valid English words out of $100$ generated "words" (based on the given English dictionary).

```
Ell - Valid
Se - Valid
Calangen - Invalid
In - Valid
Ding - Valid
Pllumest - Invalid
Oall - Invalid
Prurilll - Invalid
Uny - Invalid
Mat - Valid
Gote - Valid
Da - Valid
Pr - Valid
Ale - Valid
Me - Valid
Nor - Valid
Thecanleshm - Invalid
Adin - Valid
Wrernthe - Invalid
Ofonven - Invalid
Indecesh - Invalid
Ye - Valid
Hotesh - Invalid
Sthtaith - Invalid
Distica - Invalid
Kieshes - Invalid
Alive - Valid
Ar - Valid
Aresiof - Invalid
Un - Valid
Re - Valid
Appacur - Invalid
Matarviedes - Invalid
Vachemby - Invalid
Gonshache - Invalid
No - Valid
Yer - Valid
Angr - Invalid
Quslll - Invalid
Honuthoponintretag - Invalid
Of - Valid
Prs - Valid
Re - Valid
Hereshitongrang - Invalid
Lanteshath - Invalid
Hort - Valid
Vomprshe - Invalid
Horsirs - Invalid
Of - Valid
Jond - Invalid
Andur - Invalid
Noitothallererered - Invalid
Reacudithe - Invalid
Und - Invalid
Dilll - Invalid
Faler - Invalid
Unthe - Invalid
Yege - Invalid
Juricr - Invalid
Der - Valid
Yesshivit - Invalid
Kishe - Invalid
Yer - Valid
Wendgery - Invalid
Quthrizeich - Invalid
Nat - Valid
Quthe - Invalid
Viowe - Invalid
Gere - Valid
Jr - Valid
Jul - Invalid
Stomf - Invalid
Prus - Invalid
Qunue - Invalid
Man - Valid
Stiroithiof - Invalid
Bacha - Invalid
Prered - Invalid
Hores - Invalid
Ye - Valid
Me - Valid
No - Valid
Votit - Invalid
Fe - Valid
Lesusite - Invalid
Re - Valid
Wid - Valid
Vofteillanfr - Invalid
Yes - Valid
Lallende - Invalid
Carchon - Invalid
Ofrtouf - Invalid
Vil - Valid
Quntatare - Invalid
Daseg - Invalid
Fowiovio - Invalid
Qubeay - Invalid
Morshed - Invalid
Lansed - Invalid
Patexcegne - Invalid

Number of valid words = 38
Number of invalid words = 62
```
---

# Two Letter Prediction

For **two-letter prediction**, the first step is to generate a $n \times n \times n$ tensor, $T$, such that we can perform two letter predictions to generate random "words".

$T$ is defined such that $T_{i,j, k}$ defines the probability of the next character being $k$ given the current character is $j$ and the previous character is $i$. $T$ has the following properties

- $T_{i,j, k} \in [0,1]$.
- The sum of each row ($T_{i, j}$) is one.

Using the definition of the Conditional Probability, the probability of the triplet of $(i, j, k)$ - character $i$ followed by character $j$ followed by character $k$ - is defined as

$$T_{i,j,k} = P[K\;|\;(I\cap J)] = \frac{P[K\cap (I\cap J)]}{P[I \cap J]}$$

where $I$ is the event of character $i$ appearing as the 1st character, $J$ is the event of character $j$ appearing as the 2nd character, and $K$ is the event of character $k$ appearing as the 3rd character.

$P[K\cap (I\cap J)]$ and $P[I \cap J]$ are defined as

$$P[K\cap (I\cap J)] = \frac{f_{(i,j,k)}}{|A|}$$
$$P[I\cap J] = \frac{f_{(i,j)}}{|A|}$$

where $|A|$ is the total number of characters is the US Constitution and $f_x$ is the frequency of occurence of character or sequence $x$.

Simplifying for $P[J\;|\;I]$,

$$P[K\;|\;(I\cap J)] = \frac{f_{(i,j,k)}}{f_{(i,j)}}.$$

Thus, we see that

$$T_{i,j, k} = \frac{f_{(i,j,k)}}{f_{(i,j)}}.$$

A numpy array (i.e. matrix) $f$ is initialized with a size equal to $n \times n \times n$, with each element $f_{(i,j,k)} = 0$. As the character array $A$ is iterated over, each iterated triplet of elements ($A_x$, $A_{x+1}$, and $A_{x+2}$), $1$ is added to $f_{(i, j, k)}$ in which $i$, $j$, $k$ corresponds to the indices of $A_x$, $A_{x+1}$, and $A_{x+2}$, respectively.

$f_{(i, j)}$ is equivalent to $T_{i, j, *}$, and thus $T$ can be calculating using numpy's `divide` function to divie each element of $f$ by $f_{(i,j)}$.

The contents of $T$ (with truncation) are printed to console.

In [7]:
# Calculate the count of character k following i, j within array / matrix A.
# For the last character x+1 = len(A) and the 2nd to last character x+2 = len(A),
# we will assume that it is followed by spaces.
f = np.zeros((n, n, n))

for x in range(len(A)):

    i = D[A[x]]

    if (x + 1) >= len(A):
        j = D[" "]  # Index of space character
    else:
        j = D[A[x+1]]

    if (x + 2) >= len(A):
        k = D[" "]  # Index of space character
    else:
        k = D[A[x+2]]

    f[i, j, k] = f[i, j, k] + 1

T = np.zeros((n, n, n))

# Turn each value into a probability rather than a count. Numpy's divide
# function does a true divison of the inputs elementwise, excluding instances
# in which the sum of a row in f is equal to 0.
for i in range(len(C)):
    for j in range(len(C)):
        if np.sum(f[i, j, :]) != 0:
            T[i, j, :] = f[i, j, :] / np.sum(f[i, j, :])

print('T =', T)

T = [[[0.0295858  0.00591716 0.         ... 0.         0.         0.        ]
  [1.         0.         0.         ... 0.         0.         0.        ]
  [1.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.5        0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.  

Similar to part 1, a while loop is created to simulate the system as a Markov chain starting with an arbitrary capital letter and continues until it gets a space or any other non-alpha character (i.e. punctuations and numbers).

Array `c_letters` which contains all of the capital letters that appear within the US Consitution - which, by observation, includes all letters except "X" and "Z".

The resulting words are saved to an array `words`.

The first step to creating a 'word' is to determine a random capital letter, using `random.choice`, and save to the variable `current_letter`. Because our Markov chain is a two-letter prediction, a `previous_letter` must be defined for this initial condition. The `previous_letter` is initialized as a space.

Next, the while loop is entered and iterates as long as `current_letter` is an alpha character.

Inside the while loop, the loop first appends the `current_letter` to the string `word` string, which will be the output of the while loop. A new letter is determined using `np.random.choice`; `np.random.choice` is capable of choosing a random character given a probability of occurence. In this case, the probability of occurence is defined by the row $T_{(i, j)}$ where $i$ corresponds to the index of `current_letter` and $j$ corresponds to the index of `previous_letter`.

The `current_letter` is overwritten as the next iteration's `previous_letter`; the `new_letter` is overwritten as the next iteration's `current_letter`.

Once a `new_letter` is found that is not an alpha character, the while loop breaks and saves `word` to the array `words`.

This process is wrapped by a for loop to produce $100$ random 'words'. These words are outputted to display.

In [8]:
# Array of all the capital letters that appear within the US Constitution
c_letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J",
             "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T",
             "U", "V", "W", "Y"]

words = []

# Enter a for 100-iteration loop to generate 100 random 'words'.

for i in range(100):
    # Select a random letter from the array c_letters and set our previous
    # letter a space.

    # Iterate over the while loop until a non-alpha character is reached.
    current_letter = random.choice(c_letters)
    previous_letter = " "
    word = ""

    while current_letter.isalpha():
        # Add our current letter / character to our word. Select a new random letter using
        # our probability matrix Q.
        word = word + current_letter
        new_letter = C[np.random.choice(
            len(D), 1, p=T[D[previous_letter], D[current_letter], :])][0]

        # Save the current letter to our previous letter, and
        # set the new letter as the current letter.
        previous_letter = current_letter
        current_letter = new_letter

    words.append(word)

In [9]:
# Keep a count of all valid and invalid words when checking
# against the english dictionary.
valid_count = 0
invalid_count = 0

# Iterate over all the generated words and check if they exist within
# our english dictionary array. If they exist, then output as valid
# and add to valid counter. If not, then output as invalid and add
# to invalid counter. Print results of counters to console.
for word in words:
    if word.upper() in [a.upper() for a in en_dict]:
        print(word, '- Valid')
        valid_count = valid_count + 1
    else:
        print(word, '- Invalid')
        invalid_count = invalid_count + 1

print('\nNumber of valid words =', valid_count)
print('Number of invalid words =', invalid_count)

Yeasolatiand - Invalid
Att - Valid
Presies - Invalid
Blost - Invalid
Juding - Invalid
Was - Valid
To - Valid
Secom - Invalid
Sectice - Invalid
Mond - Invalid
Unitne - Invalid
Fulate - Invalid
Fords - Valid
Quall - Invalid
Hourres - Invalid
Ame - Valid
Grand - Valid
Rect - Valid
Rect - Valid
Hugh - Valid
Milis - Invalid
Quannevesed - Invalid
Lete - Valid
Staters - Valid
Legislavicandin - Invalid
Objective - Valid
In - Valid
Vichommited - Invalid
Reprit - Invalid
Thin - Valid
Yeasever - Invalid
Aut - Invalid
Mody - Valid
Yeas - Valid
Quor - Valid
Exed - Invalid
Quare - Valid
Reasesesinated - Invalid
Goves - Invalid
Numbection - Invalid
Buting - Invalid
Prichompern - Invalid
Laws - Valid
Statersong - Invalid
Represidecles - Invalid
Gove - Valid
Quartion - Invalid
New - Valid
Prequor - Invalid
Legisharsoeve - Invalid
No - Valid
Leakent - Invalid
Affirdiame - Invalid
Reprommite - Invalid
Rufulay - Invalid
The - Valid
They - Valid
King - Valid
Offich - Invalid
Thinal - Invalid
Ambe - Valid
H

In a sample test run, below is the resulting output of the two-letter word prediction. The two-letter word prediction was able to generate $44$ valid English words out of $100$ generated "words" (based on the given English dictionary).

```
Yeasolatiand - Invalid
Att - Valid
Presies - Invalid
Blost - Invalid
Juding - Invalid
Was - Valid
To - Valid
Secom - Invalid
Sectice - Invalid
Mond - Invalid
Unitne - Invalid
Fulate - Invalid
Fords - Valid
Quall - Invalid
Hourres - Invalid
Ame - Valid
Grand - Valid
Rect - Valid
Rect - Valid
Hugh - Valid
Milis - Invalid
Quannevesed - Invalid
Lete - Valid
Staters - Valid
Legislavicandin - Invalid
Objective - Valid
In - Valid
Vichommited - Invalid
Reprit - Invalid
Thin - Valid
Yeasever - Invalid
Aut - Invalid
Mody - Valid
Yeas - Valid
Quor - Valid
Exed - Invalid
Quare - Valid
Reasesesinated - Invalid
Goves - Invalid
Numbection - Invalid
Buting - Invalid
Prichompern - Invalid
Laws - Valid
Statersong - Invalid
Represidecles - Invalid
Gove - Valid
Quartion - Invalid
New - Valid
Prequor - Invalid
Legisharsoeve - Invalid
No - Valid
Leakent - Invalid
Affirdiame - Invalid
Reprommite - Invalid
Rufulay - Invalid
The - Valid
They - Valid
King - Valid
Offich - Invalid
Thinal - Invalid
Ambe - Valid
Houses - Valid
Unites - Valid
Feleas - Invalid
Rebed - Valid
Souvert - Invalid
Facesent - Invalid
Con - Valid
Reprohis - Invalid
Impointinfent - Invalid
Bres - Invalid
Vothe - Invalid
Cas - Invalid
Con - Valid
Art - Valid
Objectionse - Invalid
Arts - Valid
If - Valid
Hamon - Invalid
If - Valid
Duthervites - Invalid
This - Valid
Fraor - Invalid
Law - Valid
Jand - Invalid
Claw - Valid
Faill - Invalid
Fithendin - Invalid
Impecorciesid - Invalid
Quor - Valid
For - Valid
Vothempeas - Invalid
If - Valid
Unit - Valid
We - Valid
Juresemed - Invalid
Offiedispearoplislaingrebtathe - Invalid
Wargerfen - Invalid
Kin - Valid
Parthe - Invalid

Number of valid words = 44
Number of invalid words = 56
```
---

# Word Prediction

For **word prediction**, an assumption is made to use a one-word approach - similar to part 1 of this solution. The first step is to generate a Stochastic Matrix $Y$ such that single word predictions can be made to generate random sentences.

$Y$ is defined such that $Y_{i,j}$ defines the probability of the next word being $j$ given the current word is $i$. As $Y$ is a Stochastic Matrix, it is given that

- $Y_{i,j} \in [0,1]$.
- The sum of each row is one.

Using the definition of the Conditional Probability, the probability of the pairing of $(i, j)$ - word $i$ following by word $j$ - is defined as

$$Y_{i,j} = P[J\;|\;I] = \frac{P[I\cap J]}{P[I]}$$

where $I$ is the event of word $i$ appearing as the 1st word and $J$ is the event of word $j$ appearing as the 2nd word.

$P[I\cap J]$ and $P[I]$ are defined as

$$P[I\cap J] = \frac{f_{(i,j)}}{|W|}$$
$$P[I] = \frac{f_{i}}{|W|}$$

where $|W|$ is the total number of words is the US Constitution and $f_x$ is the frequency of occurrence of a word or words $x$.

Simplifying for $P[J\;|\;I]$,

$$P[J\;|\;I] = \frac{f_{(i,j)}}{f_{i}}.$$

Thus, we see that

$$Y_{i,j} = \frac{f_{(i,j)}}{f_{i}}.$$

In order to associate each character with the indices in $F$ (or other matrices), a dictionary $D$ can be created that associates each unique word $x$ with an index $i_x$. This dictionary is defined as

$$ D\,[x] = i_x.$$

A numpy array (i.e. matrix) $f$ is initialized with a size equal to $m \times m$, with each element $f_{(i,j)} = 0$, where $m$ is equal to the number of unique words in the US Constitution. As the word array $W$ is iterated over, each iterated element $W_x$ and succeeding element $W_{x+1}$, $1$ is added to $f_{(i, j)}$ in which $i$ and $j$ corresponds to the indices of $W_x$ and $W_{x+1}$, respectively.

$f_i$ is equivalent to the sum of the $ith$ row of $f$, and thus $Y$ can be calculating using numpy's `divide` function to divie each element of $f$ by summation of its corresponding row.

The contents of $Y$ (with truncation) are printed to console.

In [10]:
# Create a dictionary / hash of the set X - this dictionary D can
# provide the index within our matrix Y. We can do this by calling
# D[word], in which 'word' is our word of interest.

D = {k: v for v, k in enumerate(X)}

# Calculate the count of character j following i within array / matrix J.
# Skip the last word as it should be a period.
m = len(X)
f = np.zeros((m, m))

for x in range(len(W) - 1):
    i = D[W[x]]
    j = D[W[x+1]]

    f[i, j] = f[i, j] + 1

# Turn each value into a probability rather than a count. Numpy's divide
# function does a true divison of the inputs elementwise, excluding instances
# in which the sum of a row in f is equal to 0.
Y = np.divide(f, f.sum(axis=1)[:, None],
              out=np.zeros_like(f),
              where=f.sum(axis=1)[:, None] != 0)

print('Y =', Y)

Y = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Similar to part 1, a while loop is created to simulate the system as a Markov chain starting with an arbitrary word and continues until it reaches a period - thus ending the sentence. The resulting sentence is saved to a string sentence.

The first step to creating a 'word' is to determine a word, using `random.choice`, and save to the variable `current_word`. Next, the while loop is entered and iterates as long as `current_word` is not a period.

Inside the while loop, the loop first appends the `current_word` to the string `sentence` string, which will be the output of the while loop. A new letter is determined using `np.random.choice`; `np.random.choice` is capable of choosing a random word given a probability of occurence. In this case, the probability of occurence is defined by the row $Y_{i}$ where $i$ corresponds to the index of `current_word`.

This `new_word` is appended onto the sentence with a space after the previous word. There are few special conditions in which a space is not added (for punctuations).

Once a `new_word` is found that is a period, the while loop breaks. The resulting sentence is outputted to display.

In [11]:
# Select a random word from the array X (which contains all unique words
# in the US consitution.) Append this word onto our sentence and enter
# the while loop. Iterate over the while loop until a period is reached.
current_word = random.choice(X)
sentence = current_word

while current_word != ".":
    # Select a random word using our Probability matrix Y as generated before.

    current_word = X[np.random.choice(len(D), 1, p=Y[D[current_word], :])][0]

    # If the current word is a punctuation, do not add a space before the 'word'.
    # Else, add a space to the end of our current sentence before adding the word.

    if current_word == "." or current_word == "," or current_word == ")" or current_word == ":":
        sentence = sentence + current_word
    else:
        sentence = sentence + " " + current_word

print(sentence)

longer Term of the Laws of the President shall originate in the same, they shall not exceeding ten, or property be made within that Office of citizens twenty-one years of the House, both Houses shall be vested in Proportion to make any other State and Vice-President of Choosing Senators and other Place than once.


In a sample test run, the following sentence was generated:

```
longer Term of the Laws of the President shall originate in the same, they shall not exceeding ten, or property be made within that Office of citizens twenty-one years of the House, both Houses shall be vested in Proportion to make any other State and Vice-President of Choosing Senators and other Place than once.
```